# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 23 2022 8:30AM,240253,10,0085795554,ISDIN Corporation,Released
1,Jun 23 2022 8:30AM,240253,10,0085795576,ISDIN Corporation,Released
2,Jun 23 2022 8:30AM,240253,10,0085795578,ISDIN Corporation,Released
3,Jun 23 2022 8:30AM,240253,10,0085795579,ISDIN Corporation,Released
4,Jun 23 2022 8:30AM,240253,10,0085795580,ISDIN Corporation,Released
5,Jun 23 2022 8:30AM,240253,10,0085795581,ISDIN Corporation,Released
6,Jun 23 2022 8:30AM,240253,10,0085795582,ISDIN Corporation,Released
7,Jun 23 2022 8:30AM,240253,10,0085795583,ISDIN Corporation,Released
8,Jun 23 2022 8:30AM,240253,10,0085795585,ISDIN Corporation,Released
9,Jun 23 2022 8:30AM,240253,10,0085795584,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
27,240251,Executing,9
28,240251,Released,8
29,240253,Released,29
30,240254,Released,1
31,240255,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240249,NaN,NaN,2.0
240251,NaN,9.0,8.0
240253,NaN,NaN,29.0
240254,NaN,NaN,1.0
240255,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0.0,0.0,1.0
240147,0.0,0.0,1.0
240161,0.0,0.0,1.0
240194,0.0,0.0,1.0
240197,0.0,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0,0,1
240147,0,0,1
240161,0,0,1
240194,0,0,1
240197,0,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,0,0,1
1,240147,0,0,1
2,240161,0,0,1
3,240194,0,0,1
4,240197,0,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,,,1
1,240147,,,1
2,240161,,,1
3,240194,,,1
4,240197,,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 23 2022 8:30AM,240253,10,ISDIN Corporation
29,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc."
30,Jun 22 2022 4:41PM,240254,22,"NBTY Global, Inc."
31,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd"
48,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc."
50,Jun 22 2022 2:59PM,240241,16,Seqirus
51,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America
52,Jun 22 2022 2:46PM,240237,16,Seqirus
53,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America
54,Jun 22 2022 2:30PM,240233,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 23 2022 8:30AM,240253,10,ISDIN Corporation,,,29
1,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",,,1
2,Jun 22 2022 4:41PM,240254,22,"NBTY Global, Inc.",,,1
3,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",,9,8
4,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",,,2
5,Jun 22 2022 2:59PM,240241,16,Seqirus,,,1
6,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,,,1
7,Jun 22 2022 2:46PM,240237,16,Seqirus,,,1
8,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America,,,1
9,Jun 22 2022 2:30PM,240233,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:30AM,240253,10,ISDIN Corporation,29,,
1,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",1,,
2,Jun 22 2022 4:41PM,240254,22,"NBTY Global, Inc.",1,,
3,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",8,9,
4,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2,,
5,Jun 22 2022 2:59PM,240241,16,Seqirus,1,,
6,Jun 22 2022 2:51PM,240239,20,Sartorius Stedim North America,1,,
7,Jun 22 2022 2:46PM,240237,16,Seqirus,1,,
8,Jun 22 2022 2:37PM,240235,20,Sartorius Stedim North America,1,,
9,Jun 22 2022 2:30PM,240233,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:30AM,240253,10,ISDIN Corporation,29,,
1,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",1,,
2,Jun 22 2022 4:41PM,240254,22,"NBTY Global, Inc.",1,,
3,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",8,9,
4,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:30AM,240253,10,ISDIN Corporation,29.0,NaN,NaN
1,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",1.0,NaN,NaN
2,Jun 22 2022 4:41PM,240254,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",8.0,9.0,NaN
4,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 8:30AM,240253,10,ISDIN Corporation,29.0,0.0,0.0
1,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",1.0,0.0,0.0
2,Jun 22 2022 4:41PM,240254,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Jun 22 2022 3:52PM,240251,19,"AdvaGen Pharma, Ltd",8.0,9.0,0.0
4,Jun 22 2022 3:43PM,240249,19,"GUSA Granules USA, Inc.",2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1201077,33.0,0.0,0.0
15,240205,0.0,1.0,0.0
16,720672,3.0,0.0,0.0
19,1681443,18.0,9.0,7.0
20,1201101,21.0,23.0,2.0
21,480408,1.0,1.0,0.0
22,480472,2.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1201077,33.0,0.0,0.0
1,15,240205,0.0,1.0,0.0
2,16,720672,3.0,0.0,0.0
3,19,1681443,18.0,9.0,7.0
4,20,1201101,21.0,23.0,2.0
5,21,480408,1.0,1.0,0.0
6,22,480472,2.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,0.0,0.0
1,15,0.0,1.0,0.0
2,16,3.0,0.0,0.0
3,19,18.0,9.0,7.0
4,20,21.0,23.0,2.0
5,21,1.0,1.0,0.0
6,22,2.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,15,Released,0.0
2,16,Released,3.0
3,19,Released,18.0
4,20,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,0.0,0.0,7.0,2.0,0.0,0.0
Executing,0.0,1.0,0.0,9.0,23.0,1.0,0.0
Released,33.0,0.0,3.0,18.0,21.0,1.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,7.0,2.0,0.0,0.0
1,Executing,0.0,1.0,0.0,9.0,23.0,1.0,0.0
2,Released,33.0,0.0,3.0,18.0,21.0,1.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,0.0,0.0,7.0,2.0,0.0,0.0
1,Executing,0.0,1.0,0.0,9.0,23.0,1.0,0.0
2,Released,33.0,0.0,3.0,18.0,21.0,1.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()